In [13]:
import torch
import numpy as np
import glob
import os
from tqdm import tqdm
import csv
import random
import h5py
import sys
sys.path.append('/Users/theomaetz/Downloads/MA3/INL/Setup/lighthouse-main')

from torch.profiler import profile, record_function, ProfilerActivity

import torch.nn.functional as F
import scipy.io as sio

from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.nn import global_max_pool
from torch_geometric.data import Data
import lightning as L

from torch_geometric.loader import DataLoader

import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR

import wandb

import matplotlib.pyplot as plt
import time

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data


from torchmetrics.classification import MulticlassF1Score

In [14]:
from pytorch_lightning.loggers import WandbLogger

wandb.login()
wandb.init()

wandb_logger = WandbLogger()


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [15]:
y = np.load('/Users/theomaetz/Downloads/MA3/INL/Setup/dataset/label.npy')
X_train =  np.load('/Users/theomaetz/Downloads/MA3/INL/Setup/dataset/X_train.npy')



In [16]:
X_train.shape

(23224, 31, 1770)

In [18]:
X_train = X_train.reshape(-1,93,590)

In [20]:
dataset = []

for idx in tqdm(range(X_train.shape[0])):
    
    eeg_clip = X_train[idx,:,:]
    
        
    label = y[idx]

    if label == 7:
        continue

    dataset.append( ( torch.tensor(eeg_clip).float(), 
                    torch.tensor((label)  , dtype=torch.long) ) ) 
                      
    
random.shuffle(dataset)   

100%|██████████| 23224/23224 [00:58<00:00, 393.75it/s]


In [ ]:
class AttentionBlock(nn.Module):
    def __init__(self, embed_dim, hidden_dim, num_heads, dropout=0.0):
        """Attention Block.

        Args:
            embed_dim: Dimensionality of input and attention feature vectors
            hidden_dim: Dimensionality of hidden layer in feed-forward network
                         (usually 2-4x larger than embed_dim)
            num_heads: Number of heads to use in the Multi-Head Attention block
            dropout: Amount of dropout to apply in the feed-forward network
        """
        super().__init__()

        self.layer_norm_1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads)
        self.layer_norm_2 = nn.LayerNorm(embed_dim)
        self.linear = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, embed_dim),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        inp_x = self.layer_norm_1(x)
        x = x + self.attn(inp_x, inp_x, inp_x)[0]
        x = x + self.linear(self.layer_norm_2(x))
        return x

In [21]:
train_dataloader = DataLoader(dataset , batch_size = 64  )

In [22]:
class AttentionBlock(nn.Module):
    def __init__(self, embed_dim, hidden_dim, num_heads, dropout=0.0):
        """Attention Block.

        Args:
            embed_dim: Dimensionality of input and attention feature vectors
            hidden_dim: Dimensionality of hidden layer in feed-forward network
                         (usually 2-4x larger than embed_dim)
            num_heads: Number of heads to use in the Multi-Head Attention block
            dropout: Amount of dropout to apply in the feed-forward network
        """
        super().__init__()

        self.layer_norm_1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads)
        self.layer_norm_2 = nn.LayerNorm(embed_dim)
        self.linear = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, embed_dim),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        inp_x = self.layer_norm_1(x)
        x = x + self.attn(inp_x, inp_x, inp_x)[0]
        x = x + self.linear(self.layer_norm_2(x))
        return x

In [38]:
class VisionTransformer(nn.Module):
    def __init__(
        self,
        embed_dim,
        hidden_dim,
        num_t_pints,
        num_heads,
        num_layers,
        num_classes,
        num_patches,
        dropout=0.0,
    ):
        """Vision Transformer.

        Args:
            embed_dim: Dimensionality of the input feature vectors to the Transformer
            hidden_dim: Dimensionality of the hidden layer in the feed-forward networks
                         within the Transformer
            num_channels: Number of channels of the input (3 for RGB)
            num_heads: Number of heads to use in the Multi-Head Attention block
            num_layers: Number of layers to use in the Transformer
            num_classes: Number of classes to predict
            patch_size: Number of pixels that the patches have per dimension
            num_patches: Maximum number of patches an image can have
            dropout: Amount of dropout to apply in the feed-forward network and
                      on the input encoding
        """
        super().__init__()


        
        self.input_layer = nn.Linear(num_t_pints  , embed_dim)
        self.transformer = nn.Sequential(
            *(AttentionBlock(embed_dim, hidden_dim, num_heads, dropout=dropout) for _ in range(num_layers))
        )
        self.mlp_head = nn.Sequential(nn.LayerNorm(embed_dim), nn.Linear(embed_dim, num_classes))
        self.dropout = nn.Dropout(dropout)

        # Parameters/Embeddings
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.pos_embedding = nn.Parameter(torch.randn(1, 1 + num_patches, embed_dim))

    def forward(self, x):
        # Preprocess input
        
        #x = torch.log( torch.abs( torch.fft.fft(x , dim=-1) ) + 1e-10 )

        x = torch.cat( [torch.log(torch.abs( torch.fft.fft(x , dim=-1).imag[:,:,0:295]))  , torch.log(torch.abs(torch.fft.fft(x , dim=-1).real[:,:,0:295]) ) ] , dim=-1)

        x = (x - x.mean(dim=-1 , keepdim=True) )/x.std(dim=-1 , keepdim=True) 

        B, T, _ = x.shape
        x = self.input_layer(x)

        # Add CLS token and positional encoding
        cls_token = self.cls_token.repeat(B, 1, 1)
        x = torch.cat([cls_token, x], dim=1)
        x = x + self.pos_embedding[:, : T + 1]

        # Apply Transforrmer
        x = self.dropout(x)
        x = x.transpose(0, 1)
        x = self.transformer(x)

        # Perform classification prediction
        cls = x[0]
        out = self.mlp_head(cls)
        return out

In [39]:
class ViT(L.LightningModule):
    def __init__(self, model_kwargs):
        super().__init__()
    
        self.model = VisionTransformer(**model_kwargs)


    def forward(self, x):
        return self.model(x)

    def configure_optimizers(self):
    
        optimizer = optim.Adam(params=self.parameters(),
                            lr = 1e-3)
        
        scheduler = CosineAnnealingLR(optimizer, T_max = 50)
        
        return [optimizer], [{"scheduler": scheduler, "interval": "epoch"}]


    def training_step(self, batch, batch_idx):
        imgs, labels = batch
        preds = self.model(imgs)
        loss = F.cross_entropy(preds, labels)

        # wandb.log({ "Bridge-GPT-Loss": loss})

        return loss


In [40]:
model_kwargs={
        "embed_dim": 64,
        "hidden_dim": 64,
        "num_heads": 8,
        "num_layers": 4,
        "num_t_pints": 590,
        "num_patches": 93,
        "num_classes": 7,
        "dropout": 0.2,
    }

model = ViT(model_kwargs)

In [41]:
trainer = L.Trainer(max_epochs= 50 , devices= 1, accelerator="cpu")

trainer.fit(model, train_dataloader )

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type              | Params | Mode 
----------------------------------------------------
0 | model | VisionTransformer | 145 K  | train
----------------------------------------------------
145 K     Trainable params
0         Non-trainable params
145 K     Total params
0.581     Total estimated model params size (MB)
51        Modules in train mode
0         Modules in eval mode


Epoch 1:  36%|███▌      | 129/359 [01:11<02:06,  1.81it/s, v_num=3]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined